In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob


from mpl_toolkits.mplot3d import Axes3D  #3Dplot
import seaborn as sns
import japanize_matplotlib

In [2]:
# names にNANの列名を入れておかないとバグる
col_names = ['エリア','発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時','NAN1']

df_stop =pd.read_csv('../../../卒論関連書類/データ/電源停止データ/outages_20230111143516.csv',encoding='CP932',names=col_names)[1:]
df_stop= df_stop.drop(columns='NAN1')
# 沖縄を除く
df_stop_drop_okinawa =df_stop[df_stop['エリア'] != '沖縄']

# 時間だけKingから持ってくる
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})

In [43]:
# df_stop_king を作成する 20230112修正版

# names にNANの列名を入れておかないとバグる
col_names = ['エリア','発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時','NAN1']

df_stop =pd.read_csv('../../../卒論関連書類/データ/電源停止データ/outages_20230111143516.csv',encoding='CP932',names=col_names)[1:]
df_stop= df_stop.drop(columns='NAN1')
# 沖縄を除く
df_stop_drop_okinawa =df_stop[df_stop['エリア'] != '沖縄']

# 時間だけKingから持ってくる
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})

df_stop['停止・低下ユニット数'] = 1
df_stop.loc[df_stop['低下量'].isna(),['低下量']] = df_stop[df_stop['低下量'].isna()]['認可出力']


# 停止日時をTimeStampに変換
stop_str_datelist = df_stop['停止日時'].values
str_datelist = stop_str_datelist
stop_datetime_list = []
for str_date in str_datelist:

    stop_datetime_list.append(dt.datetime.strptime(str_date,'%Y/%m/%d %H:%M'))

    if stop_datetime_list[-1].year>2050:
        stop_datetime_list[-1] = dt.datetime.strptime('2050/12/31 23:30','%Y/%m/%d %H:%M')
        # print('yes',stop_datetime_list[-1],len(stop_datetime_list))

df_stop['停止日時'] = stop_datetime_list

# 復旧日時をTimeStampに変換,復旧日時では終日止まっていることとしている。
df_stop['復旧予定日'] =df_stop['復旧予定日'].fillna('NONE')
start_str_datelist = df_stop['復旧予定日'].values
str_datelist = start_str_datelist
start_datetime_list = []

for str_date in str_datelist:
    if str_date=='NONE':
        start_datetime = dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M')
    else:
        start_datetime = dt.datetime.strptime(f'{str_date} 23:30','%Y/%m/%d %H:%M')



    start_datetime_list.append(start_datetime)
    # if stop_datetime_list[-1].year>2061:
        # start_datetime_list[-1] = dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M')
        # print('yes',start_datetime_list[-1],len(stop_datetime_list))
new_start_datetimelist =[]
for date in start_datetime_list:
    if date.year >2060:
        new_start_datetimelist.append(dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M'))
    else:
        new_start_datetimelist.append(date)        


df_stop['復旧予定日'] = new_start_datetimelist

df_stop['停止日時'] = pd.to_datetime(df_stop['停止日時'])
df_stop['復旧予定日'] = pd.to_datetime(df_stop['復旧予定日'])
# 2018年4月1日以降に停止したもの　と　2018年4月1日以前に停止して2018年4月1日から2022年12月31日の間に復旧した
date_start = dt.datetime.strptime('2018/04/01','%Y/%m/%d')
df_stop_select = df_stop[(df_stop['停止日時']>=date_start) | ((df_stop['復旧予定日']>=date_start) &(df_stop['復旧予定日'].dt.year<=2022) & (df_stop['停止日時']<=date_start))]




# まとめるファイルの初期設定
df_stop_king = pd.DataFrame(columns=['DateTime','認可出力合計','低下量合計','停止・低下ユニット数合計'])
df_stop_king['DateTime'] =df_king['DateTime']
# それぞれのパラメータの値を０にする。その後足し算ができるように
df_stop_king = df_stop_king.fillna(0)

for num in range(len(df_stop_select)):

    df_select = df_stop_select[num:num+1]
    start_point =df_select['停止日時'].values[0]
    end_point = df_select['復旧予定日'].values[0]

    col_names_stop_king = ['認可出力','低下量','停止・低下ユニット数']
    try:
        for k in range(len(col_names_stop_king)):
            df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),[f'{col_names_stop_king[k]}合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)][f'{col_names_stop_king[k]}合計'].values + float(df_select[f'{col_names_stop_king[k]}'].values[0])
    except:
        df = df_select
        print(df_select)


/var/folders/c5/xmh5z7yd191cl82bdh1n_kzc0000gn/T/ipykernel_11874/3730365638.py:87: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),[f'{col_names_stop_king[k]}合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)][f'{col_names_stop_king[k]}合計'].values + float(df_select[f'{col_names_stop_king[k]}'].values[0])
/var/folders/c5/xmh5z7yd191cl82bdh1n_kzc0000gn/T/ipykernel_11874/3730365638.py:87: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.is

In [47]:
df_stop_king.to_csv('../../../卒論関連書類/データ/電源停止データ/電源停止データ整理済み.csv',index=False)

In [48]:
df_stop_king.to_csv('../../../卒論関連書類/データ/抽出＿分析データ/電源停止合算/電源停止データ修正版20230112.csv',index=False)

In [ ]:
# df_stop_king を作成する

# names にNANの列名を入れておかないとバグる
col_names = ['エリア','発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時','NAN1']

df_stop =pd.read_csv('../../../卒論関連書類/データ/電源停止データ/outages_20230111143516.csv',encoding='CP932',names=col_names)[1:]
df_stop= df_stop.drop(columns='NAN1')
# 沖縄を除く
df_stop_drop_okinawa =df_stop[df_stop['エリア'] != '沖縄']

# 時間だけKingから持ってくる
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})

df_stop['停止・低下ユニット数'] = 1
df_stop.loc[df_stop['低下量'].isna(),['低下量']] = df_stop[df_stop['低下量'].isna()]['認可出力']


# 停止日時をTimeStampに変換
stop_str_datelist = df_stop['停止日時'].values
str_datelist = stop_str_datelist
stop_datetime_list = []
for str_date in str_datelist:

    stop_datetime_list.append(dt.datetime.strptime(str_date,'%Y/%m/%d %H:%M'))

    if stop_datetime_list[-1].year>2050:
        stop_datetime_list[-1] = dt.datetime.strptime('2050/12/31 23:30','%Y/%m/%d %H:%M')
        # print('yes',stop_datetime_list[-1],len(stop_datetime_list))

df_stop['停止日時'] = stop_datetime_list

# 復旧日時をTimeStampに変換,復旧日時では終日止まっていることとしている。
df_stop['復旧予定日'] =df_stop['復旧予定日'].fillna('NONE')
start_str_datelist = df_stop['復旧予定日'].values
str_datelist = start_str_datelist
start_datetime_list = []

for str_date in str_datelist:
    if str_date=='NONE':
        start_datetime = dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M')
    else:
        start_datetime = dt.datetime.strptime(f'{str_date} 23:30','%Y/%m/%d %H:%M')



    start_datetime_list.append(start_datetime)
    # if stop_datetime_list[-1].year>2061:
        # start_datetime_list[-1] = dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M')
        # print('yes',start_datetime_list[-1],len(stop_datetime_list))
new_start_datetimelist =[]
for date in start_datetime_list:
    if date.year >2060:
        new_start_datetimelist.append(dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M'))
    else:
        new_start_datetimelist.append(date)        


df_stop['復旧予定日'] = new_start_datetimelist


# まとめるファイルの初期設定
df_stop_king = pd.DataFrame(columns=['DateTime','認可出力合計','低下量合計','停止・低下ユニット数合計'])
df_stop_king['DateTime'] =df_king['DateTime']
# それぞれのパラメータの値を０にする。その後足し算ができるように
df_stop_king = df_stop_king.fillna(0)

for num in range(len(df_stop)):

    df_select = df_stop[num:num+1]
    start_point =df_select['停止日時'].values[0]
    end_point = df_select['復旧予定日'].values[0]

    col_names_stop_king = ['認可出力','低下量','停止・低下ユニット数']
    try:
        for k in range(len(col_names_stop_king)):
            df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),[f'{col_names_stop_king[k]}合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)][f'{col_names_stop_king[k]}合計'].values + float(df_select[f'{col_names_stop_king[k]}'].values[0])
    except:
        df = df_select
        print(df_select)


In [44]:
df_stop_king

,DateTime,認可出力合計,低下量合計,停止・低下ユニット数合計
0,2018-04-01 00:00:00,53484820.0,53484820.0,126.0
1,2018-04-01 00:30:00,104552890.0,102677262.0,255.0
2,2018-04-01 01:00:00,102012890.0,100007262.0,255.0
3,2018-04-01 01:30:00,102012890.0,100007262.0,255.0
4,2018-04-01 02:00:00,102012890.0,100007262.0,255.0
...,...,...,...,...
83902,2023-01-05 21:30:00,57207370.0,57207370.0,157.0
83903,2023-01-05 22:00:00,57207370.0,57207370.0,157.0
83904,2023-01-05 22:30:00,57207370.0,57207370.0,157.0
83905,2023-01-05 23:00:00,57632370.0,57632370.0,159.0


In [42]:
df

NameError: name 'df' is not defined

In [39]:
# 2018年4月1日以降に停止したもの　と　2018年4月1日以前に停止して2018年4月1日から2022年12月31日の間に復旧したもの
date_start = dt.datetime.strptime('2018/04/01','%Y/%m/%d')
df_stop_select = df_stop[(df_stop['停止日時']>=date_start) | ((df_stop['復旧予定日']>=date_start) &(df_stop['復旧予定日'].dt.year<=2022) & (df_stop['停止日時']<=date_start))]

df_stop_select.to_csv('../../../卒論関連書類/データ/電源停止データ/電源停止20180401整形.csv')

In [40]:
df_stop_select[300:]

,エリア,発電事業者,発電所コード,発電所名,発電形式,ユニット名,認可出力,停止区分,種別,低下量,停止日時,復旧見通し,復旧予定日,停止原因,最終更新日時,停止・低下ユニット数
301,東京,電源開発,31804,電源開発磯子火力発電所新１号機,火力（石炭）,1号機,600000,出力低下,低下・設備故障,350000,2022-12-31 20:38:00,あり,2023-01-04 23:30:00,TPA-22-185\r\nA脱硫塔リフレッシュ運転\r\n最大抑制量：350000kW\r...,2022/12/20 15:42,1
302,東京,電源開発,31804,電源開発磯子火力発電所新１号機,火力（石炭）,1号機,600000,出力低下,低下・設備故障,200000,2022-12-29 00:12:00,あり,2022-12-31 23:30:00,TPA-22-184\r\nB脱硫塔リフレッシュ運転\r\n最大抑制量：200000kW\r...,2022/12/20 15:40,1
303,東京,電源開発,31804,電源開発磯子火力発電所新１号機,火力（石炭）,1号機,600000,出力低下,低下・設備故障,350000,2022-12-24 01:22:00,あり,2022-12-25 23:30:00,TPA-22-183\r\nA脱硫塔リフレッシュ運転\r\n最大抑制量：350000kW\r...,2022/12/20 15:36,1
304,東京,広野ＩＧＣＣパワー合同会社,327W1,広野ＩＧＣＣ発電所,火力（石炭）,単独,543000,出力低下,低下・設備故障,136000,2022-12-18 13:00:00,あり,2022-12-21 23:30:00,ガス火炉関連設備,2022/12/20 14:07,1
305,中国,電源開発,75202,電源開発竹原火力発電所３号機,火力（石炭）,3号機,700000,出力低下,低下・その他,360000,2023-01-28 22:00:00,あり,2023-01-30 23:30:00,"TRA-22-030\r\n最大抑制量：360,000kw\r\n最小抑制量：360,000...",2022/12/20 13:59,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26048,沖縄,沖縄電力株式会社,09202,金武2号,火力（石炭）,2号機,220000,計画停止,NaN,220000,2018-04-04 00:00:00,あり,2018-05-25 23:30:00,NaN,2017/01/23 17:35,1
26094,関西,株式会社コベルコパワー神戸,5115664F60,神戸発電所,火力（石炭）,２号機,700000,計画停止,NaN,700000,2019-03-09 01:00:00,あり,2019-03-11 23:30:00,NaN,2017/01/11 17:29,1
26848,東京,君津共同火力株式会社,6065330508,君津共同火力株式会社君津共同発電所,火力（ガス）,4号機,350000,計画停止,NaN,350000,2016-09-01 06:00:00,あり,2019-03-31 23:30:00,定検,2016/09/01 04:12,1
27572,四国,土佐発電株式会社,81562,土佐発電所,火力（石炭）,単独,167000,計画停止,NaN,167000,2018-10-27 00:00:00,あり,2018-10-31 23:30:00,NaN,2016/04/21 16:38,1


In [32]:
df_king

,DateTime,intra_price(円/kWh),spot_price(円/kWh),gap_price[intra-spot](円/kWh),intra_volume（MWh/h）,spot_volume(kWh),北電太陽光想定(MW),北電風力想定(MW),北電太陽光実績(MW),北電風力実績(MW),...,東京予備率(%),中部予備率(%),北陸予備率(%),関西予備率(%),中国予備率(%),四国予備率(%),九州予備率(%),認可出力合計(kW),低下量合計(kW),停止・低下ユニット数合計
0,2018-04-01 00:00:00,7.62,7.30,0.32,58.8,6098500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88445020.0,88445020.0,177.0
1,2018-04-01 00:30:00,7.52,6.93,0.59,54.0,6225500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55206600.0,55206600.0,98.0
2,2018-04-01 01:00:00,7.50,6.78,0.72,57.3,6356500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55206600.0,55206600.0,98.0
3,2018-04-01 01:30:00,7.56,6.84,0.72,63.0,6439500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55206600.0,55206600.0,98.0
4,2018-04-01 02:00:00,6.89,6.92,-0.03,61.6,6439000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55206600.0,55206600.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83902,2023-01-05 21:30:00,NaN,21.29,NaN,NaN,22544600,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124166830.0,114507230.0,293.0
83903,2023-01-05 22:00:00,NaN,21.86,NaN,NaN,22180900,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153219770.0,148365820.0,387.0
83904,2023-01-05 22:30:00,NaN,21.86,NaN,NaN,22143700,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128895980.0,119756777.0,301.0
83905,2023-01-05 23:00:00,NaN,21.74,NaN,NaN,22054200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117378280.0,108640230.0,280.0


In [34]:
date_start = dt.datetime.strptime('2018/04/01','%Y/%m/%d')

In [ ]:
# df_stop_select = df_stop[(df_stop['停止日時'].dt.year>=2018) | ((df_stop['復旧予定日'].dt.year>=2018) &(df_stop['復旧予定日'].dt.year<=2022) & (df_stop['停止日時'].dt.year<=2018))]